In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.imports import *
from fastai.conv_learner import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [ ]:
import csv
import json
from shutil import copyfile
from random import *

PATH = 'data/imaterialist/'
arch = resnet34
sz=256

img_dict = {}
json_data=open(f'{PATH}test.json').read()
data = json.loads(json_data)["images"]
for node in data:
    img_dict[node['image_id']] = str(node['url'])

tests = os.listdir(f'{PATH}test')

for file in tests:
    del img_dict[int(file.split('.')[0])]

print(len(img_dict))

# use random image if certain tests not exist
for k,v in img_dict.items():
    #DownloadImage (k,v)
    copyfile(f'{PATH}test/'+tests[random(0,len(tests)-1)], f'{PATH}test/'+str(k)+'.jpg')
    
print (len(os.listdir(f'{PATH}test')))

label_csv = f'{PATH}train_v2.csv'
n = len(list(open(label_csv)))-1
val_idxs = get_cv_idxs(n)

def get_data(sz):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_basic, max_zoom=1.3)
    return ImageClassifierData.from_csv(PATH, 'train', label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name='test')

data = get_data(sz)
learn = ConvLearner.pretrained(arch, data)
learn.load(f'{34}{sz}-2')

In [ ]:
log_preds,y = learn.TTA()
print (data.test_ds.fnames[1], y[1])
print (len(data.test_ds.fnames))
import csv

def export_test_result(files, y):
    with open(f'{PATH}submission.csv', 'w') as csvfile:
        fieldnames = ['id','predicted']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for idx, file in enumerate(files):
            try:
                writer.writerow({'id': file.split('/')[1].split('.')[0], 'predicted': y[idx]})
                del img_dict[file.split('/')[1].split('.')[0]]
            except Exception as ex:
                print (ex)

export_test_result(data.test_ds.fnames, y)